In [113]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn import neural_network, preprocessing
from sklearn.preprocessing import StandardScaler
np.random.seed(42)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
pd.set_option('max_columns', None)

In [114]:
def design_df_basic(df):
    # booleans
    df['is_first_booking'] = df['is_first_booking'].astype(int)
    df['is_user_logged_in'] = df['is_user_logged_in'].astype(int)
    df[df.request_nonesmoke.isnull()] = 0
    df[df.request_earlycheckin.isnull()] = 0
    df[df.request_highfloor.isnull()] = 0
    df[df.request_largebed.isnull()] = 0
    df[df.request_twinbeds.isnull()] = 0
    df[df.request_airport.isnull()] = 0
    df[df.request_latecheckin.isnull()] = 0
    
    df = handle_date(df, 'booking_datetime')
    df = handle_date(df, 'checkout_date')
    df = handle_date(df, 'checkin_date')
    df = handle_date(df, 'cancellation_datetime')
    
    # columns dropped
    df.drop(['h_booking_id',
             'hotel_id',
             'hotel_live_date',
             'h_customer_id',
             'language',
             'hotel_city_code',
             'guest_nationality_country_name'
             ], axis=1, inplace=True)
     
    
    # get date differences
    df['sub_checkout_checkin'] = (df['checkout_date'].dt.round('1d') - df['checkin_date'].dt.round('1d')) / np.timedelta64(1, 'D')
    df['sub_checkin_booking'] = (df['checkin_date'].dt.round('1d') - df['booking_datetime'].dt.round('1d')) / np.timedelta64(1, 'D')
    df[df['sub_checkin_booking'] == -1] = 0

    # add feature to check if booking includes weekend
    df['bus_days'] = np.busday_count(df['checkin_date'].values.astype('datetime64[D]'), df['checkout_date'].values.astype('datetime64[D]'))
    df['weekends'] = df['sub_checkout_checkin'] - df['bus_days']

    return df

In [115]:
def handle_date(df, col_name):
    df[col_name] = df[col_name].apply(
        lambda x: np.nan if x == '0000-00-00' or x == np.nan else x)
    df[col_name] = pd.to_datetime(df[col_name], errors='coerce')
    if col_name != 'cancellation_datetime':
        df[col_name + '_month'] = pd.to_numeric(df[col_name].dt.month)
        df[col_name + '_date_day'] = pd.to_numeric(df[col_name].dt.day)
        df[col_name + '_weekday'] = pd.to_numeric(df[col_name].dt.weekday)
        df[col_name + '_year'] = pd.to_numeric(df[col_name].dt.year)
        df[col_name + '_quarter'] = pd.to_numeric(df[col_name].dt.quarter)
    return df

In [116]:
DUMMIES = ["original_payment_type", "original_payment_method", "charge_option",
           "hotel_brand_code", "hotel_chain_code", "hotel_area_code", "accommadation_type_name",
           "original_payment_currency", "customer_nationality", "origin_country_code",
           "cancellation_policy_code", "hotel_country_code"]

In [117]:
def design_matrix_using_one_hot_encoding(df):
    """
    Create a design matrix from a dataframe using one-hot encoding.
    """
    df = design_df_basic(df)
    for item in DUMMIES:
        df = pd.concat([df, pd.get_dummies(df[item], prefix=item)], axis=1)
        df.drop(item, axis=1, inplace=True)        
    return df


In [118]:
def design_matrix_using_label_encoding(df):
    le_original_payment_type = preprocessing.LabelEncoder()
    le_original_payment_type.fit(df.original_payment_type)
    df.original_payment_type = le_original_payment_type.transform(df.original_payment_type.values)

    le_original_payment_method = preprocessing.LabelEncoder()
    le_original_payment_method.fit(df.original_payment_method)
    df.original_payment_method = le_original_payment_method.transform(df.original_payment_method.values)

    le_charge_option = preprocessing.LabelEncoder()
    le_charge_option.fit(df.charge_option)
    df.charge_option = le_charge_option.transform(df.charge_option.values)

    le_hotel_brand_code = preprocessing.LabelEncoder()
    le_hotel_brand_code.fit(df.hotel_brand_code)
    df.hotel_brand_code = le_hotel_brand_code.transform(df.hotel_brand_code.values)

    le_hotel_chain_code = preprocessing.LabelEncoder()
    le_hotel_chain_code.fit(df.hotel_chain_code)
    df.hotel_chain_code = le_hotel_chain_code.transform(df.hotel_chain_code.values)

    le_hotel_area_code = preprocessing.LabelEncoder()
    le_hotel_area_code.fit(df.hotel_area_code)
    df.hotel_area_code = le_hotel_area_code.transform(df.hotel_area_code.values)

    le_accommadation_type_name = preprocessing.LabelEncoder()
    le_accommadation_type_name.fit(df.accommadation_type_name)
    df.accommadation_type_name = le_accommadation_type_name.transform(df.accommadation_type_name.values)

    le_original_payment_currency = preprocessing.LabelEncoder()
    le_original_payment_currency.fit(df.original_payment_currency)
    df.original_payment_currency = le_original_payment_currency.transform(df.original_payment_currency.values)

    le_customer_nationality = preprocessing.LabelEncoder()
    le_customer_nationality.fit(df.customer_nationality)
    df.customer_nationality = le_customer_nationality.transform(df.customer_nationality.values)

    le_cancellation_policy_code = preprocessing.LabelEncoder()
    le_cancellation_policy_code.fit(df.cancellation_policy_code)
    df.cancellation_policy_code = le_cancellation_policy_code.transform(df.cancellation_policy_code.values)

    le_origin_country_code = preprocessing.LabelEncoder()
    le_origin_country_code.fit(df.origin_country_code)
    df.origin_country_code = le_origin_country_code.transform(df.origin_country_code.values)

    le_hotel_country_code = preprocessing.LabelEncoder()
    le_hotel_country_code.fit(df.hotel_country_code)
    df.hotel_country_code = le_hotel_country_code.transform(df.hotel_country_code.values)

    # booleans
    df['is_first_booking'] = df['is_first_booking'].astype(int)
    df['is_user_logged_in'] = df['is_user_logged_in'].astype(int)
    df[df.request_nonesmoke.isnull()] = 0
    df[df.request_earlycheckin.isnull()] = 0
    df[df.request_highfloor.isnull()] = 0
    df[df.request_largebed.isnull()] = 0
    df[df.request_twinbeds.isnull()] = 0
    df[df.request_airport.isnull()] = 0
    df[df.request_latecheckin.isnull()] = 0

    # columns dropped
    df.drop(['h_booking_id',
             'hotel_id',
             'hotel_live_date',
             'h_customer_id',
             'language',
             'hotel_city_code',
             'guest_nationality_country_name'
             ], axis=1, inplace=True)
     
    
    # get date differences
    df['sub_checkout_checkin'] = (df['checkout_date'].dt.round('1d') - df['checkin_date'].dt.round('1d')) / np.timedelta64(1, 'D')
    df['sub_checkin_booking'] = (df['checkin_date'].dt.round('1d') - df['booking_datetime'].dt.round('1d')) / np.timedelta64(1, 'D')
    df[df['sub_checkin_booking'] == -1] = 0

    # add feature to check if booking includes weekend
    df['bus_days'] = np.busday_count(df['checkin_date'].values.astype('datetime64[D]'), df['checkout_date'].values.astype('datetime64[D]'))
    df['weekends'] = df['sub_checkout_checkin'] - df['bus_days']

    # clean up
    # df.drop(['booking_datetime', 'checkout_date', 'checkin_date'], axis=1, inplace=True)

    # df['sub_cancel_checkin'] = (df['cancellation_date'].dt.round('1d') - df['checkin_date'].dt.round('1d')) / np.timedelta64(1, 'D')

    return df

In [119]:
def train_classifier_model(X, Y):
    # normalize data
    scaler = StandardScaler().fit(X)
    X = scaler.transform(X)

    #fit model
    model = sklearn.neural_network.MLPClassifier(random_state=1, max_iter=500, hidden_layer_sizes=(50,10))
    model.fit(X, Y)
    
    # model = neural_network.MLPRegressor

    return model

def train_regressor_model(X, Y):
    # normalize data
    scaler = StandardScaler().fit(X)
    X = scaler.transform(X)

    #fit model
    model = sklearn.neural_network.MLPRegressor(random_state=1, max_iter=500, hidden_layer_sizes=(50,10))
    model.fit(X, Y)
    
    # model = neural_network.MLPRegressor

    return model

In [120]:
train_set = pd.read_csv("../datasets/agoda_cancellation_train.csv")
train_set.head()

,h_booking_id,booking_datetime,checkin_date,checkout_date,hotel_id,hotel_country_code,hotel_live_date,hotel_star_rating,accommadation_type_name,charge_option,h_customer_id,customer_nationality,guest_is_not_the_customer,guest_nationality_country_name,no_of_adults,no_of_children,no_of_extra_bed,no_of_room,origin_country_code,language,original_selling_amount,original_payment_method,original_payment_type,original_payment_currency,is_user_logged_in,cancellation_policy_code,is_first_booking,request_nonesmoke,request_latecheckin,request_highfloor,request_largebed,request_twinbeds,request_airport,request_earlycheckin,cancellation_datetime,hotel_area_code,hotel_brand_code,hotel_chain_code,hotel_city_code
0,-9223194055642672935,2018-06-28 21:15:00,2018-07-09 00:00:00,2018-07-13 00:00:00,6452,HK,2009-06-28 02:02:00,4.0,Hotel,Pay Now,6003835896819550002,China,0,China,2,0,0,1,CN,S.Chinese / Mainland,267.55,UNKNOWN,Invoice,CNY,False,365D100P_100P,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1192,NaN,NaN,142
1,-9222713784330706132,2018-08-10 22:31:00,2018-08-16 00:00:00,2018-08-17 00:00:00,47729,CN,2011-06-07 11:52:00,4.0,Hotel,Pay Later,6337727081989900092,South Africa,0,South Africa,2,0,0,1,ZA,English,114.15,Visa,Credit Card,ZAR,False,1D1N_1N,False,1.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,5617,902.0,600.0,1273
2,-9222411208325704942,2018-09-14 07:55:00,2018-09-14 00:00:00,2018-09-15 00:00:00,780431,KR,2014-11-20 15:43:00,4.0,Hotel,Pay Now,4569478218455470035,South Korea,0,South Korea,2,0,0,1,KR,Korean,115.79,MasterCard,Credit Card,KRW,False,2D1N_1N,True,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,3819,NaN,NaN,2799
3,-9222220845872895471,2018-06-25 07:33:00,2018-07-02 00:00:00,2018-07-03 00:00:00,291365,JP,2011-11-21 12:27:00,3.0,Hotel,Pay Later,5223266395530630065,Singapore,0,Singapore,2,0,0,1,SG,English / United Kingdom,68.74,American Express,Credit Card,SGD,True,1D1N_100P,False,1.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,2379,466.0,236.0,1448
4,-9221127186162682116,2018-07-23 10:06:00,2018-08-09 00:00:00,2018-08-10 00:00:00,479046,TH,2013-06-06 10:04:00,3.0,Resort,Pay Later,2879166343687280017,Thailand,0,Thailand,8,0,0,2,TH,Thai,127.76,MasterCard,Credit Card,THB,False,14D100P_100P,True,1.0,0.0,0.0,1.0,0.0,0.0,0.0,2018-07-23,1506,NaN,NaN,616


In [121]:
train_set.columns

Index(['h_booking_id', 'booking_datetime', 'checkin_date', 'checkout_date',
       'hotel_id', 'hotel_country_code', 'hotel_live_date',
       'hotel_star_rating', 'accommadation_type_name', 'charge_option',
       'h_customer_id', 'customer_nationality', 'guest_is_not_the_customer',
       'guest_nationality_country_name', 'no_of_adults', 'no_of_children',
       'no_of_extra_bed', 'no_of_room', 'origin_country_code', 'language',
       'original_selling_amount', 'original_payment_method',
       'original_payment_type', 'original_payment_currency',
       'is_user_logged_in', 'cancellation_policy_code', 'is_first_booking',
       'request_nonesmoke', 'request_latecheckin', 'request_highfloor',
       'request_largebed', 'request_twinbeds', 'request_airport',
       'request_earlycheckin', 'cancellation_datetime', 'hotel_area_code',
       'hotel_brand_code', 'hotel_chain_code', 'hotel_city_code'],
      dtype='object')

In [122]:
labels = train_set['cancellation_datetime'].isnull().astype(int).replace({0:1, 1:0})
labels.head()

0    0
1    0
2    0
3    0
4    1
Name: cancellation_datetime, dtype: int64

In [123]:
features = design_matrix_using_one_hot_encoding(train_set)
features.head()

,booking_datetime,checkin_date,checkout_date,hotel_star_rating,guest_is_not_the_customer,no_of_adults,no_of_children,no_of_extra_bed,no_of_room,original_selling_amount,is_user_logged_in,is_first_booking,request_nonesmoke,request_latecheckin,request_highfloor,request_largebed,request_twinbeds,request_airport,request_earlycheckin,cancellation_datetime,booking_datetime_month,booking_datetime_date_day,booking_datetime_weekday,booking_datetime_year,booking_datetime_quarter,checkout_date_month,checkout_date_date_day,checkout_date_weekday,checkout_date_year,checkout_date_quarter,checkin_date_month,checkin_date_date_day,checkin_date_weekday,checkin_date_year,checkin_date_quarter,sub_checkout_checkin,sub_checkin_booking,bus_days,weekends,original_payment_type_0,original_payment_type_Credit Card,original_payment_type_Gift Card,original_payment_type_Invoice,original_payment_method_0,original_payment_method_7Eleven,original_payment_method_ATM Bersama,original_payment_method_Affin Online,original_payment_method_Alfamart,original_payment_method_Alipay,original_payment_method_AmOnline,original_payment_method_American Express,original_payment_method_Apple Pay,original_payment_method_BAY,original_payment_method_BBL,original_payment_method_Bank Islam,original_payment_method_Big C,original_payment_method_CIMB Clicks Malaysia,original_payment_method_Family Mart,original_payment_method_Hong Leong Connect,original_payment_method_IDEAL,original_payment_method_Indomaret,original_payment_method_JCB,original_payment_method_K PLUS,original_payment_method_KTB,original_payment_method_MasterCard,original_payment_method_MayBank2U,original_payment_method_PBe,original_payment_method_PayPal,original_payment_method_Prima,original_payment_method_RHB Now,original_payment_method_SCB,original_payment_method_Tesco Lotus,original_payment_method_Thanachart,original_payment_method_UNKNOWN,original_payment_method_UnionPay - Creditcard,original_payment_method_UnionPay - Debit Card,original_payment_method_Visa,original_payment_method_WeChat,original_payment_method_mPay,charge_option_0,charge_option_Pay Later,charge_option_Pay Now,charge_option_Pay at Check-in,hotel_brand_code_0.0,hotel_brand_code_1.0,hotel_brand_code_2.0,hotel_brand_code_3.0,hotel_brand_code_4.0,hotel_brand_code_6.0,hotel_brand_code_7.0,hotel_brand_code_8.0,hotel_brand_code_9.0,hotel_brand_code_10.0,hotel_brand_code_12.0,hotel_brand_code_13.0,hotel_brand_code_14.0,hotel_brand_code_15.0,hotel_brand_code_17.0,hotel_brand_code_18.0,hotel_brand_code_20.0,hotel_brand_code_21.0,hotel_brand_code_22.0,hotel_brand_code_23.0,hotel_brand_code_24.0,hotel_brand_code_25.0,hotel_brand_code_26.0,hotel_brand_code_27.0,hotel_brand_code_28.0,hotel_brand_code_29.0,hotel_brand_code_30.0,hotel_brand_code_31.0,hotel_brand_code_32.0,hotel_brand_code_33.0,hotel_brand_code_35.0,hotel_brand_code_39.0,hotel_brand_code_40.0,hotel_brand_code_41.0,hotel_brand_code_42.0,hotel_brand_code_43.0,hotel_brand_code_44.0,hotel_brand_code_45.0,hotel_brand_code_46.0,hotel_brand_code_47.0,hotel_brand_code_49.0,hotel_brand_code_50.0,hotel_brand_code_51.0,hotel_brand_code_52.0,hotel_brand_code_53.0,hotel_brand_code_54.0,hotel_brand_code_55.0,hotel_brand_code_57.0,hotel_brand_code_58.0,hotel_brand_code_59.0,hotel_brand_code_60.0,hotel_brand_code_61.0,hotel_brand_code_62.0,hotel_brand_code_63.0,hotel_brand_code_64.0,hotel_brand_code_65.0,hotel_brand_code_66.0,hotel_brand_code_68.0,hotel_brand_code_69.0,hotel_brand_code_72.0,hotel_brand_code_73.0,hotel_brand_code_74.0,hotel_brand_code_75.0,hotel_brand_code_76.0,hotel_brand_code_78.0,hotel_brand_code_79.0,hotel_brand_code_80.0,hotel_brand_code_81.0,hotel_brand_code_82.0,hotel_brand_code_83.0,hotel_brand_code_84.0,hotel_brand_code_85.0,hotel_brand_code_86.0,hotel_brand_code_87.0,hotel_brand_code_88.0,hotel_brand_code_89.0,hotel_brand_code_90.0,hotel_brand_code_91.0,hotel_brand_code_92.0,hotel_brand_code_93.0,hotel_brand_code_94.0,hotel_brand_code_95.0,hotel_brand_code_97.0,hotel_brand_code_98.0,hotel_bra

In [124]:

features['cancellation_datetime'] = pd.to_datetime(features['cancellation_datetime'], errors='coerce')
features['checkout_date'] = pd.to_datetime(features['checkout_date'], errors='coerce')
features['checkin_date'] = pd.to_datetime(features['checkin_date'], errors='coerce')
features['booking_datetime'] = pd.to_datetime(features['booking_datetime'], errors='coerce')


In [125]:
# cancellation date - booking date
def get_cancellation_date_diff(df): 
    date_sub = (df[df['cancellation_datetime'].notna()]['cancellation_datetime'] - df[df['cancellation_datetime'].notna()]['booking_datetime'])
    # print(date_sub.dt.days)
    no_cancel = df['cancellation_datetime'].isnull().astype(int)
    no_cancel = no_cancel.replace({0:-1, 1:0})
    # print(no_cancel)
    # merge date_sub with no_cancel where date_sub is defined
    return pd.concat([no_cancel, date_sub.dt.days], axis=1).max(axis=1)


# SET LABELS
labels = get_cancellation_date_diff(features)

features.drop(['cancellation_datetime','booking_datetime', 'checkout_date', 'checkin_date'], axis=1, inplace=True)

In [126]:
(labels>0.5).sum()

4782

In [ ]:
X_train_regressor, X_test_regressor, y_train_regressor, y_test_regressor = train_test_split(
    features, labels, test_size=0.2, random_state=42)


In [127]:
X_train_classifier, X_test_classifier, y_train_classifier, y_test_classifier = train_test_split(
    features, (labels!=0).astype(int), test_size=0.2, random_state=42)

In [128]:
features.isnull().sum().sort_index()

accommadation_type_name_0                0
accommadation_type_name_Apartment        0
accommadation_type_name_Boat / Cruise    0
accommadation_type_name_Bungalow         0
accommadation_type_name_Capsule Hotel    0
                                        ..
request_nonesmoke                        0
request_twinbeds                         0
sub_checkin_booking                      0
sub_checkout_checkin                     0
weekends                                 0
Length: 6558, dtype: int64

In [129]:
y_train_regressor.sum()

87692.0

In [131]:
X = features
y = labels

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [134]:
rnd = RandomForestClassifier().fit(X_train, y_train)
y_pred = rnd.predict(X_test)

In [135]:
accuracy_score(y_test, y_pred)

0.887146266621207

In [130]:
# Training
model_regressor = train_regressor_model(X_train_regressor, y_train_regressor)
print(y_train_regressor)
model_classifier = train_classifier_model(X_train_classifier, y_train_classifier)

C:\Users\t-nweisler\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:619: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


54348    0.0
19312    0.0
12945    0.0
48989    0.0
23534    0.0
        ... 
54343    0.0
38158    0.0
860      0.0
15795    0.0
56422    0.0
Length: 46927, dtype: float64


C:\Users\t-nweisler\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:619: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


In [ ]:
# Model accuracy
y_class_pred= model_classifier.predict(X_test_classifier)
# print(f'Accuracy of classifier model: {accuracy_score(y_test_classifier, y_pred)}')
y_reg_predict = model_regressor.predict(X_test_regressor)>0.5
# accuracy of regressor model
print(f'Accuracy of regressor model: {accuracy_score(y_test_regressor, y_reg_predict)}')
# sklearn.metrics.mean_squared_error(y_test_regressor, model_regressor.predict(X_test_regressor))

print(model_regressor.predict(X_test_regressor))


Accuracy of regressor model: 0.8880838731674053
[-1013.31652247  -594.04073833  -594.04073833 ...  -635.7003263
 -9636.64985551 -3625.37044625]


In [ ]:
# Load data
train_set = pd.read_csv("../datasets/agoda_cancellation_train.csv")

# train_set.drop(['cancellation_datetime'], axis=1, inplace=True)
features = design_matrix_using_one_hot_encoding(train_set)
sub_checkout_cancelation = []
labels = features['cancellation_datetime'].isnull().astype(int)
features['cancellation_datetime'] = pd.to_datetime(features['cancellation_datetime'], errors='coerce')
for i in range(len(features)):
    if(pd.notnull(features['cancellation_datetime'][i])):
        sub_checkout_cancelation.append((features['cancellation_datetime'][i]-features['booking_datetime'][i]).dt.days)
    else:
        sub_checkout_cancelation.append(0)
   

AttributeError: 'Timedelta' object has no attribute 'dt'

In [ ]:
# train model
classifier = train_model(X_train, y_train)
y_pred = classifier.predict(X_test)
accuracy_score(y_test, y_pred)
# # How many people canceled after one week
# canceled_after_one_week = features[features['sub_checkout_checkin'] == 1]

0.7375554040231844

In [ ]:
# load predicted data
predicted = pd.read_csv("predicted_results.csv")
actual = pd.read_csv("actual_results.csv")

actual = actual['label'].str.strip().str[-1].astype(int)

predicted =  predicted.replace({0:1, 1:0})

# calculate predicted accuracy
accuracy_score(actual, predicted)


0.8571428571428571